In [3]:
import pandas as pd
import numpy as np

In [4]:
with open('/home/lab08/drama.txt', 'r') as f:
    drama = f.read()

drama = drama.split('\n\n\n')

drama_data = pd.DataFrame(data=None, columns=('제목','라벨','타이틀','조회수','날짜','좋아요','댓글수','댓글'))
for i in drama[:-1]:
    제목 = i[i.find('제목 :'): i.find('라벨 :')][len('제목 : '):].strip()
    라벨 = i[i.find('라벨 :') : i.find('타이틀 :')][len('라벨 : '):].strip()
    타이틀 = i[i.find('타이틀 :') : i.find('조회수 :')][len('타이틀 : '):].strip()
    조회수 = i[i.find('조회수 :') : i.find('날짜 : ')][len('조회수 : '):].strip()
    날짜 = i[i.find('날짜 :') : i.find('좋아요 : ')][len('날짜 : '):].strip()
    좋아요 = i[i.find('좋아요 :') : i.find('댓글수 :')][len('좋아요 : '):].strip()
    댓글수 = i[i.find('댓글수 :') : i.find('댓글 :')][len('댓글수 : '):].strip()
    댓글 = i[i.find('댓글 :') :][len('댓글 : '):].strip()
    temp = pd.DataFrame.from_dict([{'제목' : 제목,
                     '라벨' : 라벨,
                     '타이틀' : 타이틀,
                     '조회수' : 조회수,
                     '날짜'   : 날짜,
                     '좋아요' : 좋아요,
                     '댓글수' : 댓글수,
                     '댓글' : 댓글}])
    drama_data = pd.concat([drama_data,temp])


sentences = list(map(str,[i[2:-2].split('\n') for i in drama_data.댓글]))
all_tokens = []
for j in sentences:
    sent_token = j[2:-2].split(', ')
    for token in sent_token:
        all_tokens.append(token.strip()[2:-2])

all_tokens

[' 대가를 받지 않은 자발적 리뷰입니다. \\\\n최신 드라마중 몰입감 최고였습니다ㅠ\\\\n\\\\n하드코어 범죄 드라마 <어느 날>이 궁금하시다면? ',
 '한국드라마에서 주인공 누명쓰고 좆되는거 존나 많지만 이렇게 좆되는게 와닿는 드라마는 처음 봄ㅋㅋㅋ 그만큼 연기랑 연출이 다른 차원임 김수현이 진짜 쫄리게 연기를 개잘함',
 'Desde el Polo Sur disfruto de cada trabajo de Soo-Hyun!!! Y esta joya no es la excepción. Cómo siemp',
 ' ha dejado ni un solo movimiento ni emoción guardada para darle vida a "Hyun-Soo" . Estoy viendo por segunda ver el primer episodio. Para digerirlo cuadro por cuadro y no perder ni un eslabón suelto en esta historia. Me saco el sombrero ante este talentoso actor. Saludos desde Chile 🇨🇱🇨🇱🇨🇱🇨🇱',
 '김수현 연기 너무 잘함..... 매번 다른 캐릭터를 자기껄로 만들어버리네 도민준은 생각도 안남\\\\n저음 목소리 멋져......',
 '유일하게 어린 아이들과 보는 영화 리뷰 채널 이네요~\\\\n욕설이나 비방/비하 발언이 없이 내용만 리뷰 해주셔서 너무 좋게 보고 있어요~^^',
 '김수현이 우는연기 원탑인듯 맘 아프게 잘울어',
 '새벽동안 다 봤는데 이거 너무 재밌음. 연출도 세련되고 배우들 연기력도 탄탄해서 다음화가 너무 기대됨ㅠ 쿠팡 플레이가 진입장벽이 높아서 아쉬움 넷플릭스나 적어도 티빙 같은 곳에서 나왔으면 사람들이 훨씬 많이 볼 것 같은데 아쉽긴 하지만 월정액 결제까지 할 정도로 재밌단 뜻',
 '연기력 발성 지리는 김홍파배우님이 드디어 뭔가 조연이 아니라 분량 꾸준히나오는 주연급이라서 너무 좋아요!! 암살

In [11]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from bertopic import BERTopic

In [12]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [13]:
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [15]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens",
                 vectorizer_model=vectorizer,
                 nr_topics=50,
                 top_n_words=10,
                 calculate_probabilities=True)

In [16]:
topics, probs = model.fit_transform(all_tokens)

In [18]:
for i in range(len(topics)+1):
    print(model.get_topic(i))

[('00', 0.06544293982013588), ('18', 0.06190655243622175), ('13', 0.0575371637927804), ('14', 0.05753241155250641), ('20', 0.057414706408393926), ('12', 0.057168778258771966), ('17', 0.05619314494410297), ('40', 0.05068828869507938), ('11', 0.05058816524851662), ('19', 0.0496855414512386)]
[('여자', 0.02260324593741508), ('지만', 0.01850189351548191), ('드라마', 0.018461469592635754), ('사람', 0.017769551146082217), ('으로', 0.016558529824262976), ('마시', 0.016124731221866466), ('\\\\', 0.01606868392062287), ('에서', 0.014057126547375896), ('다고', 0.013844819101008201), ('는데', 0.013841941187768932)]
[('ㅋㅋㅋ', 0.12767161030033292), ('ㅋㅋ', 0.06851082636881453), ('미쳤', 0.027962658216992786), ('제목', 0.027344760693976337), ('웃기', 0.022761347908036196), ('14', 0.01981092058422219), ('냐고', 0.01818144826959641), ('진짜', 0.017282192061935778), ('미친', 0.016250316007606076), ('편집', 0.015847529850802202)]
[('김수현', 0.045449166364972246), ('한국', 0.039372107394770955), ('연기', 0.022607948853041713), ('박형식', 0.01978850